In [1]:
!pip install pandas

import os
import pandas as pd
os.chdir('../BlackbirdDatasetData')

print("done")

done


In [2]:
def find_closest_timestamp(target, timestamps):
    """
   
    """
    n = len(timestamps)
    left, right = 0, n - 1
    
    if target <= timestamps[left]:
        return left
    if target >= timestamps[right]:
        return right
    

    while left <= right:
        mid = left + (right - left) // 2
        if timestamps[mid] == target:
            return mid
        elif target < timestamps[mid]:
            right = mid - 1
        else:
            left = mid + 1
    
   
    if abs(target - timestamps[left]) < abs(target - timestamps[right]):
        return left
    else:
        return right


In [3]:
df = pd.read_csv("ampersand/yawForward/maxSpeed2p0/csv/blackbird_slash_imu.csv")
print(df.shape)

df_new = df[['rosbagTimestamp', 'x.1', 'y.1', 'z.1', 'x.2', 'y.2', 'z.2']]
df_new

(26818, 23)


,rosbagTimestamp,x.1,y.1,z.1,x.2,y.2,z.2
0,1534109177595045000,-0.008649,-0.001554,-0.004707,-0.033402,0.074667,-9.743565
1,1534109177605052000,-0.009067,-0.001133,-0.002173,-0.049817,0.076040,-9.765403
2,1534109177615351000,-0.010037,-0.000604,-0.002709,-0.039370,0.071830,-9.749911
3,1534109177625700000,-0.006735,0.000170,-0.003779,-0.053559,0.075601,-9.737576
4,1534109177638407000,-0.007260,-0.001319,-0.004831,-0.051046,0.062624,-9.761662
...,...,...,...,...,...,...,...
26813,1534109445882826000,-0.010435,-0.001710,-0.002186,0.019678,-0.117366,-9.780371
26814,1534109445892932000,-0.008345,-0.000231,-0.003669,0.013543,-0.137790,-9.761053
26815,1534109445903401000,-0.007699,-0.001405,-0.003395,-0.004269,-0.120730,-9.749159
26816,1534109445912780000,-0.010007,-0.001677,-0.002234,0.016815,-0.134791,-9.755929


In [22]:

start, end, step = 1534109177000000000, 1534109446700000000, 100000000


lol = range(start, end, step)


df_lol = pd.DataFrame({'TimeStamp': lol})

print(df_lol)



                TimeStamp
0     1534109177000000000
1     1534109177100000000
2     1534109177200000000
3     1534109177300000000
4     1534109177400000000
...                   ...
2692  1534109446200000000
2693  1534109446300000000
2694  1534109446400000000
2695  1534109446500000000
2696  1534109446600000000

[2697 rows x 1 columns]


In [29]:


df_new['rosbagTimestamp'] = df_new['rosbagTimestamp'].astype(int)


df_lol = df_lol.sort_values(by='TimeStamp')


timestamps = df_lol['TimeStamp'].values
print(timestamps)


closest_indices = [find_closest_timestamp(ts, timestamps) for ts in df_new['rosbagTimestamp'].values]


mapped_rows = df_lol.iloc[closest_indices]
df_new = df_new.reset_index(drop=True)
mapped_rows = mapped_rows.reset_index(drop=True)
df_combined = pd.concat([df_new, mapped_rows], axis=1)



[1534109177000000000 1534109177100000000 1534109177200000000 ...
 1534109446400000000 1534109446500000000 1534109446600000000]


In [24]:
grouped_means = df_combined.groupby('TimeStamp')[['x.1', 'y.1', 'z.1', 'x.2', 'y.2', 'z.2']].mean().reset_index()

In [25]:
grouped_means

,TimeStamp,x.1,y.1,z.1,x.2,y.2,z.2
0,1534109177600000000,-0.008240,-0.000890,-0.003750,-0.045179,0.072484,-9.751432
1,1534109177700000000,-0.007530,-0.000092,-0.004196,-0.044646,0.068618,-9.751208
2,1534109177800000000,-0.007983,0.000049,-0.003754,-0.046012,0.073144,-9.753008
3,1534109177900000000,-0.007705,-0.000327,-0.004149,-0.048374,0.071622,-9.757879
4,1534109178000000000,-0.007346,-0.000576,-0.003930,-0.047405,0.070944,-9.749974
...,...,...,...,...,...,...,...
2679,1534109445500000000,-0.008700,-0.000466,-0.003321,0.018066,-0.124640,-9.766142
2680,1534109445600000000,-0.008739,-0.000518,-0.003063,0.015497,-0.123617,-9.765734
2681,1534109445700000000,-0.008726,-0.000092,-0.003560,0.016663,-0.129123,-9.760668
2682,1534109445800000000,-0.009347,-0.000932,-0.003373,0.020652,-0.127796,-9.765043


In [26]:

grouped_means.to_csv('exported_dataframe.csv', index=False, encoding='utf-8')